### Question to ask:
1. when cleaning email address, I can check if each email address contain virginia to filter out all the emial that are computing id only and entered gmail instead of virginia.edu. However, for all 2 do file, there is one cell has wrong computing id, how to find that cell without hard coding?

2. while delete duplicate, the originial stata do file drop the later one(according to the start date), however, the privious row has more NA than the one we need to delete, is there any rules we need to follow when we delete duplicate?

3. how to deal with label, pandas dataframe do not have such functionality to assign a label to a cell

In [1]:
#this is the python script try to replicate the 2a stata do file
import pandas as pd
import numpy as np
from utils import cleanColumns

In [2]:
#define a function to convert columns name to intutitive column names
def convert_cols(df):
    df=df.rename({
                        'q1':'email',
                        'q4_1':'sim_fbsk',
                        'q5':'sim_txt_fbsk',
                        'q6_1':'sim_cmsk',
                        'q7':'sim_txt_cmsk',
                        'q8':'sim_txt_eth_behavior',
                        'q9':'sim_txt_eth_contribute',
                        'q10_1':'beh_fidgeting',
                        'q10_2':'beh_humming',
                        'q10_3':'beh_excitable',
                        'q10_4':'beh_inattentive',
                        'q10_5':'beh_short_attention',
                        'q10_6':'beh_quarrelsome',
                        'q10_7':'beh_acts_smart',
                        'q10_8':'beh_unpredictable',
                        'q10_9':'beh_defiant',
                        'q10_10':'beh_uncooperative',
                        'q10_11':'beh_easily_frustrated',
                        'q10_12':'beh_disturbs_others',
                        'q10_13':'beh_restless',
                        'q10_14':'beh_mood_changes',
                        'q11_1':'app_coach_stu',
                        'q12_1':'app_adjust_expect',
                        'q13_1':'app_guidance_couns',
                        'q14_1':'app_rec_sped',
                        'q15_1':'app_discp_refer',
                        'q16_1':'app_confer_stu',
                        'q17_1':'app_confer_parent',
                        'q18_1':'app_behavior_plan',
                        'q19_1':'app_challenge_work',
                        'q20_1':'app_spend_time',
                        'q21_1':'app_space_regroup',
                        'q22_1':'app_beh_manage_coach',
                        'q23_1':'app_beh_manage_teach',
                        'q24':'sim_txt_supports',
                        'q25_1':'sim_nervous',
                        'q25_2':'sim_beneficial',
                        'q25_3':'sim_worried_perform',
                        'q25_4':'sim_useful_tool',
                        'q25_5':'sim_relevant_studies',
                        'q25_6':'sim_relevant_prof',
                        'q25_7':'sim_like_use_again',
                        'q25_8':'sim_recommend',
                        'q25_9':'sim_sufficient_prep',
                        'q25_10':'sim_enough_time',
                        'q26':'sim_txt_beneficial',
                        'q27':'sim_txt_improve_exp',
                        'q28':'sim_txt_concerns' 
                       },axis=1)
    return df

In [3]:
#function that fixed the email address to correct format
def fix_email_add(string):
    result=''
    index=string.find("@")
    if index==-1:
        result=string+'@virginia.edu'
    else:
        result=string[:index]+'@virginia.edu'
    return result 

In [4]:
def format_email(data):
    data.email =data.email.str.strip()
    data.email =data.email.str.lower()
    email_fix = data.loc[~data["email"].str.contains("virginia")]
    email_fix['email']=email_fix['email'].apply(lambda x:fix_email_add(x))
    for i in email_fix.index:
        data.at[i,"email"]=email_fix.at[i,'email']
    
    

In [5]:
#create map for beh columns and sim columns
def convert_to_numeric(data,beh_map,sim_map,reverse_beh,reverse_sim):
    data.loc[:,'beh_fidgeting':'beh_mood_changes']=data.loc[:,'beh_fidgeting':'beh_mood_changes'].replace(beh_map)
    data.loc[:,'sim_nervous' :'sim_enough_time']=data.loc[:,'sim_nervous' :'sim_enough_time'].replace(sim_map)
    data.loc[:,'app_coach_stu':'app_beh_manage_teach']=data.loc[:,'app_coach_stu':'app_beh_manage_teach'].astype('float')
    data[['sim_fbsk','sim_cmsk']]=data[['sim_fbsk','sim_cmsk']].astype('float')
    
def generate_Iowa_Score_Scale(data):
    data['beh_rating_opdefiant']=data.loc[:,'beh_quarrelsome': 'beh_uncooperative'].mean(axis=1)
    data['beh_rating_impulsive']=data.loc[:,'beh_fidgeting':'beh_short_attention'] .mean(axis=1)
    data['beh_rating'] =data.loc[:,'beh_fidgeting':'beh_mood_changes'].mean(axis=1)
    #label var beh_rating_opdefiant "Iowa Connors Operational Defiant"
    #label var beh_rating_impulsive "Iowa Connors Impulsive"
    #label var beh_rating "Iowa Connors Overall"
    

In [6]:
#Reverse coding management approaches scale 
def Reverse_Approach_Scale(data,app_map):
    app_columns=[x for x in data.columns if x.startswith("app")]
    for x in app_columns:
        name_rc=x+'_rc'
        data[name_rc]=data[x].replace(app_map)

In [7]:
def Generate_App_Scale(data):
    data['manage_app_negative']= data.loc[:,'app_coach_stu':'app_discp_refer'].mean(axis=1)
    data['manage_app_positive']=data.loc[:,'app_confer_stu':'app_beh_manage_teach'].mean(axis=1)

### Post-Simulator Baseline Student Survey- Summer 2018_January 22, 2020_16.58.csv

In [8]:
#load the file from data
data = pd.read_csv("../data/Post-Simulator Baseline Student Survey- Summer 2018_January 22, 2020_16.58.csv",skiprows=[1,2])
data.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q25_4,Q25_5,Q25_6,Q25_7,Q25_8,Q25_9,Q25_10,Q26,Q27,Q28
0,2018-05-30 12:23:22,2018-05-30 12:24:48,Survey Preview,NaN,100,86,True,2018-05-30 12:24:49,R_1opCvzbgQYDLHKN,NaN,...,Somewhat agree,Strongly agree,Somewhat agree,Undecided,Somewhat disagree,Strongly disagree,Somewhat disagree,NaN,NaN,NaN
1,2018-05-30 13:30:45,2018-05-30 13:32:13,Survey Preview,NaN,100,87,True,2018-05-30 13:32:13,R_1JEm2v4BI4ajTNP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-08 07:18:04,2018-06-08 07:36:22,IP Address,199.111.228.147,100,1097,True,2018-06-08 07:36:22,R_ZKuGL3K4Mp1Gked,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Strongly agree,It felt like an actual classroom\n\nThe differ...,It was sort of hard to understand what they we...,none
3,2018-06-08 07:57:16,2018-06-08 08:09:18,IP Address,199.111.228.147,100,722,True,2018-06-08 08:09:18,R_SCWEWpvi6QkN4U9,NaN,...,Strongly agree,Somewhat agree,Somewhat agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Being able to try out certain ways of speaking...,Increased volume and vocal clarity.,None.
4,2018-06-08 08:19:08,2018-06-08 08:42:41,IP Address,199.111.228.147,100,1412,True,2018-06-08 08:42:41,R_1kMOho2B2VK5Hkr,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Thinking about wait time. I talked over the s...,I could use help developing ice breakers. I a...,None at this time


In [9]:
data.columns = cleanColumns(data.columns)
#drop unnecessary observation
data = data[data['q1'].notna()]


In [10]:
#drop unnecessary variables
unused_columns=['enddate', 'status', 'ipaddress', 'progress',
       'duration (in seconds)', 'finished', 'recordeddate', 'responseid',
       'recipientlastname', 'recipientfirstname', 'recipientemail',
       'externalreference', 'locationlatitude', 'locationlongitude',
       'distributionchannel', 'userlanguage']
data=data.drop(columns=unused_columns)


In [11]:
#rename the columns to intuitive names
data=convert_cols(data)
data.columns

Index(['startdate', 'email', 'sim_fbsk', 'affect', 'sim_cmsk', 'score',
       'rationale', 'sim_txt_eth_contribute', 'beh_fidgeting', 'beh_humming',
       'beh_excitable', 'beh_inattentive', 'beh_short_attention',
       'beh_quarrelsome', 'beh_acts_smart', 'beh_unpredictable', 'beh_defiant',
       'beh_uncooperative', 'beh_easily_frustrated', 'beh_disturbs_others',
       'beh_restless', 'beh_mood_changes', 'app_coach_stu',
       'app_adjust_expect', 'app_guidance_couns', 'app_rec_sped',
       'app_discp_refer', 'app_confer_stu', 'app_confer_parent',
       'app_behavior_plan', 'app_challenge_work', 'app_spend_time',
       'app_space_regroup', 'app_beh_manage_coach', 'app_beh_manage_teach',
       'sim_txt_supports', 'sim_nervous', 'sim_beneficial',
       'sim_worried_perform', 'sim_useful_tool', 'sim_relevant_studies',
       'sim_relevant_prof', 'sim_like_use_again', 'sim_recommend',
       'sim_sufficient_prep', 'sim_enough_time', 'sim_txt_beneficial',
       'sim_txt_improv

In [12]:
format_email(data)

<ipython-input-4-349bea6136b6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_fix['email']=email_fix['email'].apply(lambda x:fix_email_add(x))


In [13]:
#drop duplicate
i = data[((data.email == 'blt9q@virginia.edu') &( data.startdate == '2018-06-14 14:52:51'))].index
data=data.drop(i)

In [14]:
#drop startdate
data=data.drop(columns=['startdate'])

In [15]:
reverse_beh = {1:"Not at all", 2: "Just a little", 3: "Pretty much" ,4: "Very much"}
reverse_sim = {5: "Strongly agree", 4:"Somewhat agree", 3: "Undecided", 2: "Somewhat disagree", 1: "Strongly disagree"}
beh_map={"Not at all":1,  "Just a little":2,  "Pretty much":3 , "Very much":4}
sim_map={"Strongly agree":5, "Somewhat agree":4, "Undecided":3, "Somewhat disagree":2, "Strongly disagree":1}
app_map = {1:10, 2:9, 3:8,4:7,5:6,6:5,7:4,8:3,9:2,10:1}

In [16]:
convert_to_numeric(data,beh_map,sim_map,reverse_beh,reverse_sim)
generate_Iowa_Score_Scale(data)


In [17]:
Reverse_Approach_Scale(data,app_map)
Generate_App_Scale(data)

In [18]:
redo = pd.read_csv("../data/Post-Simulator Baseline Student Survey (only for Redos)- Summer 2018 - Copy_January 23, 2020_07.22.csv",skiprows=[1,2])
redo.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q15_1,Q16_1,Q17_1,Q18_1,Q19_1,Q20_1,Q21_1,Q22_1,Q23_1,Q24
0,2018-06-20 07:34:36,2018-06-20 07:36:32,1,NaN,100,116,1,2018-06-20 07:36:33,R_d6V6xXmfUqeOLW9,NaN,...,6.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,A
1,2018-06-20 14:16:39,2018-06-20 14:34:03,0,199.111.228.151,100,1044,1,2018-06-20 14:34:04,R_PNyNPjWT61T3ZG9,NaN,...,3.0,10.0,7.0,8.0,6.0,10.0,8.0,7.0,10.0,Breaking off into groups may be a good way to ...
2,2018-06-20 14:36:56,2018-06-20 14:40:54,0,199.111.228.151,100,238,1,2018-06-20 14:40:55,R_2ymCxxAGFvqhaAJ,NaN,...,1.0,5.0,4.0,6.0,6.0,8.0,9.0,7.0,7.0,might need help with how to deal with defiant ...
3,2018-06-20 14:44:08,2018-06-20 14:52:47,0,199.111.228.151,100,519,1,2018-06-20 14:52:47,R_1jUhlkPaIzNIy00,NaN,...,2.0,10.0,10.0,10.0,7.0,10.0,4.0,10.0,10.0,'- attention getter - in some classrooms i hav...
4,2018-06-20 15:03:08,2018-06-20 15:09:22,0,199.111.228.151,100,374,1,2018-06-20 15:09:23,R_1ezqC68mwIsFsx6,NaN,...,1.0,5.0,5.0,8.0,7.0,6.0,9.0,6.0,10.0,'-background knowledge on Ethan (techniques th...


In [19]:
redo.columns = cleanColumns(redo.columns)
#drop unnecessary observation
redo = redo[redo['q1'].notna()]
#drop unused columns
redo=redo.drop(columns=unused_columns)

In [20]:
#rename the columns to intuitive names
redo=convert_cols(redo)
redo.columns

Index(['startdate', 'email', 'rationale', 'sim_txt_eth_contribute',
       'beh_fidgeting', 'beh_humming', 'beh_excitable', 'beh_inattentive',
       'beh_short_attention', 'beh_quarrelsome', 'beh_acts_smart',
       'beh_unpredictable', 'beh_defiant', 'beh_uncooperative',
       'beh_easily_frustrated', 'beh_disturbs_others', 'beh_restless',
       'beh_mood_changes', 'app_coach_stu', 'app_adjust_expect',
       'app_guidance_couns', 'app_rec_sped', 'app_discp_refer',
       'app_confer_stu', 'app_confer_parent', 'app_behavior_plan',
       'app_challenge_work', 'app_spend_time', 'app_space_regroup',
       'app_beh_manage_coach', 'app_beh_manage_teach', 'sim_txt_supports'],
      dtype='object')

In [21]:
format_email(redo)
redo

,startdate,email,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,beh_inattentive,beh_short_attention,beh_quarrelsome,...,app_discp_refer,app_confer_stu,app_confer_parent,app_behavior_plan,app_challenge_work,app_spend_time,app_space_regroup,app_beh_manage_coach,app_beh_manage_teach,sim_txt_supports
1,2018-06-20 14:16:39,hnj2gq@virginia.edu,'-he had a lot of energy and was expelling it ...,'-I would want to know about his home life and...,4.0,4.0,4.0,4.0,3.0,3.0,...,3.0,10.0,7.0,8.0,6.0,10.0,8.0,7.0,10.0,Breaking off into groups may be a good way to ...
2,2018-06-20 14:36:56,gct2y@virginia.edu,"disruptive, disrespectful","bored, wants to be in a music class, maybe doe...",3.0,4.0,2.0,3.0,2.0,3.0,...,1.0,5.0,4.0,6.0,6.0,8.0,9.0,7.0,7.0,might need help with how to deal with defiant ...
3,2018-06-20 14:44:08,kby6hf@virginia.edu,'- distracted\n- disruptive\n- persistent\n- u...,'- lack of formal rules\n- have other teachers...,4.0,4.0,4.0,4.0,3.0,2.0,...,2.0,10.0,10.0,10.0,7.0,10.0,4.0,10.0,10.0,'- attention getter - in some classrooms i hav...
4,2018-06-20 15:03:08,sdr9x@virginia.edu,'-either wanting attention or can't control hi...,'--wanting attention or cant control himself (...,2.0,4.0,3.0,2.0,3.0,2.0,...,1.0,5.0,5.0,8.0,7.0,6.0,9.0,6.0,10.0,'-background knowledge on Ethan (techniques th...
5,2018-06-20 15:13:02,jms3ea@virginia.edu,'-Distracting -Uncooperative -Rude,He may have behavioral issues or attention iss...,4.0,4.0,4.0,4.0,3.0,3.0,...,5.0,10.0,9.0,8.0,7.0,9.0,6.0,7.0,10.0,'-Activities that peak student interest - One-...
6,2018-06-20 16:04:29,wmn6kc@virginia.edu,Distracted and disruptive,Possible family issues or learning impairments...,4.0,4.0,2.0,4.0,1.0,1.0,...,3.0,10.0,10.0,6.0,1.0,1.0,6.0,10.0,10.0,Additional classroom personnel.
8,2018-06-29 10:00:58,jlg2ga@virginia.edu,'-disruptive\n-short attention span\n-smart al...,I think that he was really bored. He seems to ...,4.0,4.0,4.0,4.0,1.0,1.0,...,2.0,10.0,7.0,10.0,9.0,10.0,10.0,7.0,9.0,'-fully establish classroom norms (especially ...
9,2018-06-29 10:11:39,yk5kb@virginia.edu,distracted\n,"possibly adhd or a learning disability, proble...",4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,10.0,7.0,9.0,2.0,10.0,8.0,10.0,10.0,fast paced work\nsilent activities\nstrong cla...
10,2018-06-29 10:16:14,kr2tvd@virginia.edu,'-Hyperactive\n-Distracted\n-,'-behavior at home\n-past classroom experiences\n,4.0,4.0,3.0,4.0,2.0,3.0,...,4.0,8.0,7.0,10.0,5.0,9.0,7.0,6.0,8.0,Separate areas to work\nFidget toys \n


In [22]:
#correct email id
i = redo[((redo.email == 'kr2tvd@virginia.edu'))].index
redo.at[i,'email']="krv2td@virginia.edu"

In [23]:
redo

,startdate,email,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,beh_inattentive,beh_short_attention,beh_quarrelsome,...,app_discp_refer,app_confer_stu,app_confer_parent,app_behavior_plan,app_challenge_work,app_spend_time,app_space_regroup,app_beh_manage_coach,app_beh_manage_teach,sim_txt_supports
1,2018-06-20 14:16:39,hnj2gq@virginia.edu,'-he had a lot of energy and was expelling it ...,'-I would want to know about his home life and...,4.0,4.0,4.0,4.0,3.0,3.0,...,3.0,10.0,7.0,8.0,6.0,10.0,8.0,7.0,10.0,Breaking off into groups may be a good way to ...
2,2018-06-20 14:36:56,gct2y@virginia.edu,"disruptive, disrespectful","bored, wants to be in a music class, maybe doe...",3.0,4.0,2.0,3.0,2.0,3.0,...,1.0,5.0,4.0,6.0,6.0,8.0,9.0,7.0,7.0,might need help with how to deal with defiant ...
3,2018-06-20 14:44:08,kby6hf@virginia.edu,'- distracted\n- disruptive\n- persistent\n- u...,'- lack of formal rules\n- have other teachers...,4.0,4.0,4.0,4.0,3.0,2.0,...,2.0,10.0,10.0,10.0,7.0,10.0,4.0,10.0,10.0,'- attention getter - in some classrooms i hav...
4,2018-06-20 15:03:08,sdr9x@virginia.edu,'-either wanting attention or can't control hi...,'--wanting attention or cant control himself (...,2.0,4.0,3.0,2.0,3.0,2.0,...,1.0,5.0,5.0,8.0,7.0,6.0,9.0,6.0,10.0,'-background knowledge on Ethan (techniques th...
5,2018-06-20 15:13:02,jms3ea@virginia.edu,'-Distracting -Uncooperative -Rude,He may have behavioral issues or attention iss...,4.0,4.0,4.0,4.0,3.0,3.0,...,5.0,10.0,9.0,8.0,7.0,9.0,6.0,7.0,10.0,'-Activities that peak student interest - One-...
6,2018-06-20 16:04:29,wmn6kc@virginia.edu,Distracted and disruptive,Possible family issues or learning impairments...,4.0,4.0,2.0,4.0,1.0,1.0,...,3.0,10.0,10.0,6.0,1.0,1.0,6.0,10.0,10.0,Additional classroom personnel.
8,2018-06-29 10:00:58,jlg2ga@virginia.edu,'-disruptive\n-short attention span\n-smart al...,I think that he was really bored. He seems to ...,4.0,4.0,4.0,4.0,1.0,1.0,...,2.0,10.0,7.0,10.0,9.0,10.0,10.0,7.0,9.0,'-fully establish classroom norms (especially ...
9,2018-06-29 10:11:39,yk5kb@virginia.edu,distracted\n,"possibly adhd or a learning disability, proble...",4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,10.0,7.0,9.0,2.0,10.0,8.0,10.0,10.0,fast paced work\nsilent activities\nstrong cla...
10,2018-06-29 10:16:14,krv2td@virginia.edu,'-Hyperactive\n-Distracted\n-,'-behavior at home\n-past classroom experiences\n,4.0,4.0,3.0,4.0,2.0,3.0,...,4.0,8.0,7.0,10.0,5.0,9.0,7.0,6.0,8.0,Separate areas to work\nFidget toys \n


In [24]:
#drop startdate
redo=redo.drop(columns=['startdate'])
generate_Iowa_Score_Scale(redo)
Reverse_Approach_Scale(redo,app_map)
Generate_App_Scale(redo)
redo

,email,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,beh_inattentive,beh_short_attention,beh_quarrelsome,beh_acts_smart,...,app_confer_stu_rc,app_confer_parent_rc,app_behavior_plan_rc,app_challenge_work_rc,app_spend_time_rc,app_space_regroup_rc,app_beh_manage_coach_rc,app_beh_manage_teach_rc,manage_app_negative,manage_app_positive
1,hnj2gq@virginia.edu,'-he had a lot of energy and was expelling it ...,'-I would want to know about his home life and...,4.0,4.0,4.0,4.0,3.0,3.0,2.0,...,1.0,4.0,3.0,5.0,1.0,3.0,4.0,1.0,4.0,8.250
2,gct2y@virginia.edu,"disruptive, disrespectful","bored, wants to be in a music class, maybe doe...",3.0,4.0,2.0,3.0,2.0,3.0,3.0,...,6.0,7.0,5.0,5.0,3.0,2.0,4.0,4.0,2.4,6.500
3,kby6hf@virginia.edu,'- distracted\n- disruptive\n- persistent\n- u...,'- lack of formal rules\n- have other teachers...,4.0,4.0,4.0,4.0,3.0,2.0,3.0,...,1.0,1.0,1.0,4.0,1.0,7.0,1.0,1.0,2.8,8.875
4,sdr9x@virginia.edu,'-either wanting attention or can't control hi...,'--wanting attention or cant control himself (...,2.0,4.0,3.0,2.0,3.0,2.0,2.0,...,6.0,6.0,3.0,4.0,5.0,2.0,5.0,1.0,3.6,7.000
5,jms3ea@virginia.edu,'-Distracting -Uncooperative -Rude,He may have behavioral issues or attention iss...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,...,1.0,2.0,3.0,4.0,2.0,5.0,4.0,1.0,6.6,8.250
6,wmn6kc@virginia.edu,Distracted and disruptive,Possible family issues or learning impairments...,4.0,4.0,2.0,4.0,1.0,1.0,1.0,...,1.0,1.0,5.0,10.0,10.0,5.0,1.0,1.0,5.2,6.750
8,jlg2ga@virginia.edu,'-disruptive\n-short attention span\n-smart al...,I think that he was really bored. He seems to ...,4.0,4.0,4.0,4.0,1.0,1.0,4.0,...,1.0,4.0,1.0,2.0,1.0,1.0,4.0,2.0,6.4,9.000
9,yk5kb@virginia.edu,distracted\n,"possibly adhd or a learning disability, proble...",4.0,4.0,4.0,4.0,4.0,4.0,1.0,...,1.0,4.0,2.0,9.0,1.0,3.0,1.0,1.0,6.8,8.250
10,krv2td@virginia.edu,'-Hyperactive\n-Distracted\n-,'-behavior at home\n-past classroom experiences\n,4.0,4.0,3.0,4.0,2.0,3.0,3.0,...,3.0,4.0,1.0,6.0,2.0,4.0,5.0,3.0,6.8,7.500


In [25]:
data.shape


(32, 66)

In [26]:
#merge the dataset using redo to update the datas
redo = redo.set_index('email')
data = data.set_index('email')
data.update(redo)
data=data.sort_values(by=['email'])
data.reset_index(inplace=True)


In [27]:
data

,email,sim_fbsk,affect,sim_cmsk,score,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,...,app_confer_stu_rc,app_confer_parent_rc,app_behavior_plan_rc,app_challenge_work_rc,app_spend_time_rc,app_space_regroup_rc,app_beh_manage_coach_rc,app_beh_manage_teach_rc,manage_app_negative,manage_app_positive
0,ajm8gx@virginia.edu,10.0,'-I think that delving into the background of ...,10.0,'-In disciplining Ethan I was able to make sur...,Ethan was off-task often.\nHe interrupted othe...,He may have ADD or another IEP. He may have a ...,4.0,4.0,4.0,...,1.0,1.0,1.0,5.0,1.0,2.0,1.0,1.0,8.2,9.375
1,alh8pk@virginia.edu,5.0,They seemed to have a good grasp on what made ...,NaN,I couldn't get to the discussion about class n...,"Ethan was very distracted, full of energy, and...","He may have a short attention span, possibly A...",4.0,4.0,3.0,...,3.0,2.0,2.0,4.0,4.0,3.0,2.0,1.0,5.0,8.375
2,alp7q@virginia.edu,5.0,'- I thought I did okay with going from questi...,2.0,I had a brief moment of success when Savannah ...,He was disruptive but occasionally responsive ...,I'd want to know if there was a reason why he ...,4.0,4.0,2.0,...,1.0,1.0,3.0,7.0,3.0,6.0,1.0,1.0,6.4,8.125
3,art2vr@virginia.edu,6.0,I honestly was more concerned with making sure...,7.0,I think I did a good job on this. I asked the ...,Super distracting. Every time I thought he was...,I think the guidance counselor and parent/guar...,2.0,4.0,3.0,...,1.0,1.0,1.0,6.0,1.0,8.0,1.0,1.0,5.2,8.500
4,bab7rr@virginia.edu,8.0,The students were on-task and seemed intereste...,4.0,The class provided one class expectation (to b...,"In this situation, Ethan was very off-task and...",A real student may act this way in class becau...,3.0,4.0,4.0,...,3.0,4.0,4.0,7.0,6.0,7.0,7.0,4.0,5.4,5.750
5,bbp5rr@virginia.edu,6.0,I think I definitely helped them talk and thin...,6.0,I thought I did a good job of managing to get ...,"Ethan wanted to do his own thing, and was in h...",Possible attention deficit disorders.\nEasily ...,3.0,2.0,2.0,...,2.0,4.0,2.0,6.0,5.0,7.0,6.0,3.0,3.0,6.625
6,blt9q@virginia.edu,8.0,I think the students understood the reading an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,cnr5a@virginia.edu,6.0,When I asked about the way Lisa felt when aske...,7.0,I used the way some of the students were actin...,Ethan was very disruptive and distracting. He ...,The fact that Ethan could not remain still and...,4.0,4.0,2.0,...,3.0,6.0,5.0,7.0,2.0,8.0,10.0,3.0,3.8,5.500
8,ecu2ug@virginia.edu,7.0,'-I attempted to use guiding statements to exp...,2.0,'- I wasn't able to get the group on task/when...,Ethan was disengaged and focused on his own ac...,Ethan probably has a negative relationship wit...,4.0,4.0,3.0,...,2.0,2.0,1.0,7.0,2.0,3.0,1.0,1.0,6.0,8.625
9,ema4ex@virginia.edu,7.0,I used positive feedback to reinforce their ex...,7.0,I think I handled Ethan's distracting behavior...,"He was loud, distracting, and wanted to talk t...",I think having Ethan on a bouncy ball chair wo...,3.0,3.0,2.0,...,4.0,4.0,3.0,3.0,4.0,4.0,6.0,3.0,5.0,7.125
